narrative_tfidf, tfidf_title, tfidf_keywords	XGBoost  	0.664090	0.559104	0.456817	0.467326  

In [1]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
from gensim.models import KeyedVectors
import pandas as pd
from sklearn.model_selection import cross_val_score, learning_curve
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, chi2, SelectFromModel 
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
import warnings

In [2]:
import numpy as np
np.random.seed(42)  # Set random seed for numpy

import random
random.seed(42)  # Set random seed for random module

In [3]:
df = pd.read_csv("D:\\MS DATA SCIENCE\\NLP TESE\\data\\data_processed_selected.csv")
df.head()

,date,title,reason,description,zone,narrative_tfidf,narrative_embeddings,narrative_bert,narrative_tfidf_title,narrative_bert_title,...,embeddings_title,tfidf_keywords,embeddings_keywords,events_clean,results_final,events_tfidf,events_embeddings,orgs,locs,entities
0,2021-10-11 12:26:32,CTT - Encomenda entregue danificada e com etiq...,0,Bom dia venho por este meio apresentar uma rec...,Lisboa,bom dia venho meio apresentar reclamação ctt e...,bom dia venho por este meio apresentar uma rec...,Bom dia venho por este meio apresentar uma rec...,ctt encomenda entregar danificar etiqueta envi...,ctt - encomenda entregue danificada e com etiq...,...,ctt - encomenda entregue danificada e com etiq...,apresentar reclamação caixa figura envio israe...,apresentar uma reclamação tinha caixa figura e...,venho apresentar enviei destinado feito proteg...,"[('CTT', 'B-ORGANIZACAO'), ('Israel', 'B-LOCAL...",venho apresentar enviar destinar fazer protege...,venho apresentar enviei destinado feito proteg...,"CTT, Aboboda",Israel,"CTT, Aboboda, Israel"
1,2021-08-31 23:20:15,CTT - Carta com certificado rasgado!,1,- Boa Noite!\r\n\r\n- Venho por este meio recl...,Coimbra,bom noite venho meio reclamar data ás durante ...,- boa noite!\r\n\r\n- venho por este meio recl...,- Boa Noite! Venho por este meio reclamar que ...,ctt carta certificar rasgar bom noite venho me...,ctt - carta com certificado rasgado! - Boa No...,...,ctt - carta com certificado rasgado !,lilia pimentel certificar profissional passage...,lilia pimentel certificado profissional passag...,venho reclamar deparei -me continha tem,"[('Lilia', 'B-PESSOA'), ('Pimentel', 'I-PESSOA')]",venho reclamar deparar conter,venho reclamar deparei - me continha tem,NaN,NaN,NaN
2,2021-02-01 19:08:32,CTT - Encomenda registada que nunca chegou ao ...,2,"Boa tarde,\r\nNo dia 17 de Dezembro de 2020 en...",Aveiro,bom tarde dia dezembro enviar encomenda França...,"boa tarde,\r\nno dia 00 de dezembro de 0000 en...","Boa tarde, No dia 17 de Dezembro de 2020 envie...",ctt encomenda registar nunca chegar destine o ...,ctt - encomenda registada que nunca chegou ao ...,...,ctt - encomenda registada que nunca chegou ao ...,frança correio registar frança correio encomen...,frança em correio registado frança em correio ...,enviei registado vejo chegou saiu chegou tem p...,"[('França', 'B-LOCAL'), ('Roissy', 'B-ORGANIZA...",enviar registar vejo chegar sair chegar passar...,enviei registado vejo chegou saiu chegou tem p...,Roissy,"França, ROISSY","Roissy, França, ROISSY"
3,2018-04-19 09:04:34,CTT - Correio Verde,2,"Bom dia,\n\nPretendia esclarecer uma situação ...",Lisboa,bom dia pretender esclarecer situação ocorrer ...,"bom dia,\n\npretendia esclarecer uma situação ...","Bom dia, Pretendia esclarecer uma situação que...",ctt correio verde bom dia pretender esclarecer...,"ctt - correio verde Bom dia, Pretendia esclar...",...,ctt - correio verde,pretender esclarecer situação pretender esclar...,pretendia esclarecer uma situação pretendia es...,pretendia esclarecer ocorreu desloquei dado fe...,"[('El', 'B-LOCAL'), ('Corte', 'I-LOCAL'), ('In...",pretender esclarecer ocorrer desloquei dar fec...,pretendia esclarecer ocorreu desloquei dado fe...,Correio,"El Corte Inglés, Lisboa Verde","Correio, El Corte Inglés, Lisboa Verde"
4,2019-07-04 10:20:41,"CTT - Aviso de recepção da carta registada, pr...",2,venho por este meio solicitar a vossa ajuda pa...,Porto,venho meio solicitar vosso ajudar saber onde e...,venho por este meio solicitar a vossa ajuda pa...,venho por este meio solicitar a vossa ajuda pa...,ctt aviso recepção carta registar preciso urge...,"ctt - aviso de recepção da carta registada, pr...",...,"ctt - aviso de recepção da carta registada , p...",papal centro pois preciso urgentemente pois pr...,papais para o centro pois preciso dele urgente...,venho solicitar saber onde se encontra regista...,[],venho solicitar saber onde encontrar registar ...,venho solicitar saber onde se encontra regista...,NaN,NaN,NaN


In [4]:
def preprocess_entities(row):
    if pd.isna(row)==False:
        entities = row.lower()  # Convert to lowercase
        entities = entities.replace(',', '')  # Remove commas
        entities = ' '.join(set(entities.split()))  # Convert to set to get unique values, then join back as a string
        return entities
df['entities'] = df['entities'].apply(preprocess_entities)

In [5]:
df['entities']

0                          israel ctt aboboda
1                                        None
2                               roissy frança
3        corte correio verde inglés el lisboa
4                                        None
                         ...                 
22425                unido portugal reino ctt
22426                                    None
22427                              aliexpress
22428                                  lisboa
22429                                    None
Name: entities, Length: 22430, dtype: object

In [6]:
df.columns

Index(['date', 'title', 'reason', 'description', 'zone', 'narrative_tfidf',
       'narrative_embeddings', 'narrative_bert', 'narrative_tfidf_title',
       'narrative_bert_title', 'narrative_embeddings_title', 'tfidf_title',
       'title_tfidf_nolemma', 'events', 'keywords', 'embeddings_title',
       'tfidf_keywords', 'embeddings_keywords', 'events_clean',
       'results_final', 'events_tfidf', 'events_embeddings', 'orgs', 'locs',
       'entities'],
      dtype='object')

## TF-IDF  

In [7]:
y=df['reason']
X = df[['narrative_tfidf', 'tfidf_title', 'tfidf_keywords', 'events_tfidf', 'entities']]

In [8]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Split the remaining data into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, stratify=y_test, random_state=42)

# Print the shape of each set
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

X_train shape: (15701, 5)
y_train shape: (15701,)
X_val shape: (3364, 5)
y_val shape: (3364,)
X_test shape: (3365, 5)
y_test shape: (3365,)


In [9]:
X_train_val = pd.concat([X_train, X_val], axis=0, ignore_index=True)
y_train_val = pd.concat([y_train, y_val], axis=0, ignore_index=True)

## **1. XGBoost**   

<span style="color:white; font-size:18px;background-color:red">* Default XGBoost()</span>

In [59]:
import optuna
def objective(trial):
    feature_set=['narrative_tfidf', 'tfidf_title', 'tfidf_keywords']
    X_train_val_combined = X_train_val[feature_set].apply(lambda x: ' '.join(x.fillna('').astype(str)), axis=1)
    X_test_combined = X_test[feature_set].apply(lambda x: ' '.join(x.fillna('').astype(str)), axis=1)

    # Transform the features using TF-IDF
    vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_df=0.95, min_df=2)
    X_train_val_combined_transformed = vectorizer.fit_transform(X_train_val_combined)
    X_test_combined_transformed = vectorizer.transform(X_test_combined)

    # Define the hyperparameter search space
    params = {
      'objective': 'multi:softmax',
      'num_class': 4,
      'eval_metric': 'mlogloss',
      'eta': trial.suggest_float('eta', 0.01, 0.3),
      'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
      'max_depth': trial.suggest_int('max_depth', 3, 15),
      'subsample': trial.suggest_float('subsample', 0.5, 1.0),
      'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
      'min_child_weight': trial.suggest_int('min_child_weight', 2, 15),
      'gamma': trial.suggest_float("gamma", 0, 1),
      'random_state': 42,  # Add other hyperparameters as needed
      'reg_alpha': trial.suggest_float('reg_alpha',0, 1),              # L1 regularization term on weights
      'reg_lambda': trial.suggest_float('reg_lambda',0, 1)
    }

    # Create and train the XGBoost classifier with the current set of hyperparameters
    clf = XGBClassifier(**params)
    clf.fit(X_train_val_combined_transformed, y_train_val)

    # Evaluate the classifier on the validation set
    cv = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
    f1_scores = cross_val_score(clf, X_train_val_combined_transformed, y_train_val, cv=cv, scoring='f1_macro')

    return f1_scores.mean()

In [65]:
import os
path = 'D:\\MS DATA SCIENCE\\NLP TESE\\optuna'
database_filename = 'description_title_keywords.db'

# Full path to the SQLite database file in Google Drive
database_path = os.path.join(path, database_filename)

# Set the SQLite database path to Google Drive path
storage = optuna.storages.RDBStorage(url=f'sqlite:///{database_path}')

In [66]:
study = optuna.create_study(storage=storage, direction='maximize', study_name='description_title_keywords',load_if_exists=True)
study.optimize(objective, n_trials=50)

trial = study.best_trial
print('Val Loss: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

[I 2023-07-30 01:05:39,386] A new study created in RDB with name: description_title_keywords
[I 2023-07-30 01:46:59,940] Trial 0 finished with value: 0.45076536635853826 and parameters: {'colsample_bytree': 0.8481781819826891, 'eta': 0.024914580506785078, 'gamma': 0.600195402231574, 'max_depth': 5, 'min_child_weight': 12, 'n_estimators': 692, 'reg_alpha': 0.5941382665998689, 'reg_lambda': 0.289947460352284, 'subsample': 0.5247350747360959}. Best is trial 0 with value: 0.45076536635853826.
[I 2023-07-30 02:09:46,554] Trial 1 finished with value: 0.4702032121386974 and parameters: {'colsample_bytree': 0.7612560333215546, 'eta': 0.20632398379829894, 'gamma': 0.13331133924189265, 'max_depth': 4, 'min_child_weight': 12, 'n_estimators': 458, 'reg_alpha': 0.533366102451221, 'reg_lambda': 0.4725380046619754, 'subsample': 0.8383017824580684}. Best is trial 1 with value: 0.4702032121386974.
[I 2023-07-30 02:41:03,255] Trial 2 finished with value: 0.45874055358778176 and parameters: {'colsample_b

[I 2023-07-30 13:32:47,285] Trial 21 finished with value: 0.46498990467896845 and parameters: {'colsample_bytree': 0.5144170729865112, 'eta': 0.17741818251300137, 'gamma': 0.08851852845177571, 'max_depth': 7, 'min_child_weight': 14, 'n_estimators': 454, 'reg_alpha': 0.5480955109029495, 'reg_lambda': 0.8662270341787979, 'subsample': 0.8270276074795879}. Best is trial 1 with value: 0.4702032121386974.
[I 2023-07-30 14:03:44,680] Trial 22 finished with value: 0.4655855667756022 and parameters: {'colsample_bytree': 0.5576349696425419, 'eta': 0.13554886127155324, 'gamma': 0.004737494700155026, 'max_depth': 6, 'min_child_weight': 15, 'n_estimators': 604, 'reg_alpha': 0.4781702677533959, 'reg_lambda': 0.6642925938408274, 'subsample': 0.8027146105626319}. Best is trial 1 with value: 0.4702032121386974.
[I 2023-07-30 14:40:28,405] Trial 23 finished with value: 0.4660159468529745 and parameters: {'colsample_bytree': 0.6281076885514805, 'eta': 0.17537792340810762, 'gamma': 0.12321065825265394, 'm

[I 2023-07-31 03:01:41,950] Trial 42 finished with value: 0.46417681742625055 and parameters: {'colsample_bytree': 0.8123209026960019, 'eta': 0.21322609068686285, 'gamma': 0.31018013179752324, 'max_depth': 3, 'min_child_weight': 2, 'n_estimators': 918, 'reg_alpha': 0.28260981317894973, 'reg_lambda': 0.343523971866212, 'subsample': 0.9403856352602971}. Best is trial 30 with value: 0.4720905631356927.
[I 2023-07-31 03:48:40,622] Trial 43 finished with value: 0.4703353965231631 and parameters: {'colsample_bytree': 0.7819888377930647, 'eta': 0.19061744654579565, 'gamma': 0.4600468156699826, 'max_depth': 4, 'min_child_weight': 4, 'n_estimators': 843, 'reg_alpha': 0.20998074625355836, 'reg_lambda': 0.4263354584385572, 'subsample': 0.867035302343105}. Best is trial 30 with value: 0.4720905631356927.
[I 2023-07-31 04:41:15,326] Trial 44 finished with value: 0.47057481797004774 and parameters: {'colsample_bytree': 0.7385861791254164, 'eta': 0.2204403887709136, 'gamma': 0.3096562463971734, 'max_

Val Loss: 0.4734151878989077
Best hyperparameters: {'colsample_bytree': 0.6660707896339382, 'eta': 0.1865203620258158, 'gamma': 0.4869470425232074, 'max_depth': 5, 'min_child_weight': 8, 'n_estimators': 747, 'reg_alpha': 0.2615683528029101, 'reg_lambda': 0.20706126430246635, 'subsample': 0.87307714888806}


In [ ]:
# in case it fails
# Load the existing study by providing the study name and storage URL
study = optuna.load_study(study_name='description_title_keywords', storage=storage)

# Continue optimization with additional trials
study.optimize(objective, n_trials=50)

trial = study.best_trial
print('Val Loss: {}'.format(trial.value))
print("Best hyperparameters: {}".format(trial.params))

In [ ]:
feature_set=['narrative_tfidf', 'tfidf_title', 'tfidf_keywords']
X_train_val_combined = X_train_val[feature_set].apply(lambda x: ' '.join(x.fillna('').astype(str)), axis=1)
X_test_combined = X_test[feature_set].apply(lambda x: ' '.join(x.fillna('').astype(str)), axis=1)

# Transform the features using TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_df=0.95, min_df=2)
X_train_val_combined_transformed = vectorizer.fit_transform(X_train_val_combined)
X_test_combined_transformed = vectorizer.transform(X_test_combined)


best_model = XGBClassifier(
    colsample_bytree=0.6660707896339382,
    eta=0.1865203620258158,
    gamma=0.4869470425232074,
    max_depth=5,
    min_child_weight=8,
    n_estimators=747,
    reg_alpha=0.2615683528029101,
    reg_lambda=0.20706126430246635,
    subsample=0.87307714888806
)
cv = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)
f1_scores = cross_val_score(best_model, X_train_val_combined_transformed, y_train_val, cv=cv, scoring='f1_macro')
acc_scores = cross_val_score(best_model, X_train_val_combined_transformed, y_train_val, cv=cv, scoring='accuracy')
rec_scores = cross_val_score(best_model, X_train_val_combined_transformed, y_train_val, cv=cv, scoring='recall_macro')
prec_scores = cross_val_score(best_model, X_train_val_combined_transformed, y_train_val, cv=cv, scoring='precision_macro')

print("Individual fold F1-scores:", f1_scores)
print("Mean cross-validation F1-score:", f1_scores.mean())
print("Individual Precision:", prec_scores)
print("Mean cross-validation Precision:", prec_scores.mean())
print("Individual Accuracy:", acc_scores)
# or np.mean
print("Mean cross-validation Accuracy:", acc_scores.mean())
print("Individual Recall:", rec_scores)
print("Mean cross-validation Recall:", rec_scores.mean())

In [ ]:
# Retrain on full training set using the best hyperparameters
best_model.fit(X_train_val_combined_transformed, y_train_val)

# Make predictions on the test dataset
y_pred_test = best_model.predict(X_test_combined_transformed)
print('test classification report')
print(classification_report(y_test, y_pred_test))

# Print the confusion matrix
fig, ax = plt.subplots(figsize=(7, 7))
confusion_mat = confusion_matrix(y_true=y_test, y_pred=y_pred_test, labels=[0,1,2,3])
ax.matshow(confusion_mat, cmap=plt.cm.Blues, alpha=0.3)
for i in range(confusion_mat.shape[0]):
    for j in range(confusion_mat.shape[1]):
        ax.text(x=j, y=i,s=confusion_mat[i, j], va='center', ha='center', size='xx-large')
plt.xlabel('Predictions', fontsize=10)
plt.ylabel('Actuals', fontsize=10)
plt.title('Confusion Matrix', fontsize=10)
plt.show()
print('\n')

In [ ]:
# Plot Learning Curves
def plot_learning_curve(best_model, X_train_val_combined_transformed, y_train_val, scoring):
    # Create StratifiedKFold cross-validator with 6 folds
    stratified_cv = StratifiedKFold(n_splits=6, shuffle=True, random_state=42)

    train_sizes, train_scores, val_scores = learning_curve(best_model, X_train_val_combined_transformed, y_train_val, cv=stratified_cv, 
                                                           scoring=scoring, train_sizes=np.linspace(0.1, 1.0, 10),
                                                           random_state=42)

    # Calculate mean and standard deviation of training and validation scores
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    valid_scores_mean = np.mean(val_scores, axis=1)
    valid_scores_std = np.std(val_scores, axis=1)

    # Plot learning curves
    plt.figure(figsize=(10, 6))
    plt.plot(train_sizes, train_scores_mean, label='Training score', color='blue')
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.2, color='blue')
    plt.plot(train_sizes, valid_scores_mean, label='Cross-validation score', color='red')
    plt.fill_between(train_sizes, valid_scores_mean - valid_scores_std,
                     valid_scores_mean + valid_scores_std, alpha=0.2, color='red')
    plt.xlabel('Number of Training Examples')
    plt.ylabel('{} Score'.format(scoring))
    plt.title('Learning Curves')
    plt.legend(loc='best')
    plt.grid()
    plt.show()

In [ ]:
plot_learning_curve(best_model, X_train_val_combined_transformed, y_train_val, 'f1_macro')

In [ ]:
plot_learning_curve(best_model, X_train_val_combined_transformed, y_train_val, 'accuracy')

<span style="color:white; font-size:18px;background-color:red">* Training XGBoost with Class Weighting</span>  

<span style="color:white; font-size:18px;background-color:red">* Resampling with smote</span>